In [1]:
# install keras tuner
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.9 MB/s eta 0:00:00


In [2]:
# import needed pages
import keras_tuner
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

<ipython-input-2-8a20da98ea88>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import Hyperband


In [14]:
# Load the MNIST dataset, containing handwritten digits and their labels
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Select a subset of the training data for demonstration purposes
size = -10000
x_train = x_train[:size]
x_val = x_train[size:]
y_train = y_train[:size]
y_val = y_train[size:]

# Normalize the pixel values to the range [0, 1] and add a channel dimension
x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

# Convert labels to one-hot encoding for categorical classification
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [26]:
def design_model(hp):
    # Create a Sequential model
    model = tf.keras.Sequential()

    # Convolutional layers
    for i in range(hp.Int("num_layers_conv", 1, 5)):
        # Add a convolutional layer with tunable number of filters and specified parameters
        model.add(
            layers.Conv2D(
                filters=hp.Int(f"filters{i}", min_value=32, max_value=256, step=32),
                kernel_size=(3, 3),
                activation='relu',
                padding='same'
            )
        )
        # Add a max pooling layer
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output from convolutional layers
    model.add(layers.Flatten())

    # Dense layers
    for i in range(hp.Int("num_layers", 1, 5)):
        # Add a dense layer with tunable number of units and activation function
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )

    # Add a dropout layer based on a boolean hyperparameter
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))

    # Output layer with softmax activation for classification
    model.add(layers.Dense(10, activation="softmax"))

    # Compile the model with a tunable learning rate
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


In [27]:
tuner = keras_tuner.RandomSearch(
    hypermodel=design_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="directory_name",
    project_name="Project_name",
)

In [28]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 43s]
val_accuracy: 0.9817500114440918

Best val_accuracy So Far: 0.9817500114440918
Total elapsed time: 00h 03m 00s


In [29]:
# Retrieve the top 2 models from the tuner
top_models = tuner.get_best_models(num_models=2)

# Select the best model from the top models
best_model = top_models[0]

# Build the best model using the input shape of the training data
best_model.build(input_shape=x_train.shape)

# Display a summary of the best model's architecture
print("Best Model Summary:")
best_model.summary()

Best Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (50000, 28, 28, 128)      1280      
                                                                 
 max_pooling2d (MaxPooling2  (50000, 14, 14, 128)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (50000, 25088)            0         
                                                                 
 dense (Dense)               (50000, 96)               2408544   
                                                                 
 dense_1 (Dense)             (50000, 256)              24832     
                                                                 
 dense_2 (Dense)             (50000, 192)              49344     
                                    

In [32]:
# Train the best model on the dataset
best_model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the best model on the test dataset
test_loss, test_accuracy = best_model.evaluate(x_test, y_test)

# Display the test loss and accuracy
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Epoch 1/10
1250/1250 [==============================] - 7s 6ms/step - loss: 0.0404 - accuracy: 0.9887 - val_loss: 0.0758 - val_accuracy: 0.9817
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0388 - accuracy: 0.9889 - val_loss: 0.1295 - val_accuracy: 0.9683
Epoch 3/10
1250/1250 [==============================] - 7s 6ms/step - loss: 0.0343 - accuracy: 0.9909 - val_loss: 0.0789 - val_accuracy: 0.9788
Epoch 4/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0305 - accuracy: 0.9921 - val_loss: 0.0778 - val_accuracy: 0.9806
Epoch 5/10
1250/1250 [==============================] - 8s 6ms/step - loss: 0.0270 - accuracy: 0.9926 - val_loss: 0.0869 - val_accuracy: 0.9814
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.0302 - accuracy: 0.9918 - val_loss: 0.0983 - val_accuracy: 0.9782
Epoch 7/10
1250/1250 [==============================] - 7s 6ms/step - loss: 0.0278 - accuracy: 0.9923 - val_loss: 0.0996 - val_accuracy: